task name: Implement FieldAwareFactorizationMachineWithOptions
task description: Implement the provided C# code for FieldAwareFactorizationMachineWithOptions as a dotnet interactive notebook.

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
// Number of features per field.
const int featureLength = 5;

// This class defines objects fed to the trained model.
private class DataPoint
{
    // Label.
    public bool Label { get; set; }

    // Features from the first field. Note that different fields can have
    // different numbers of features.
    [VectorType(featureLength)]
    public float[] Field0 { get; set; }

    // Features from the second field. 
    [VectorType(featureLength)]
    public float[] Field1 { get; set; }

    // Features from the thrid field. 
    [VectorType(featureLength)]
    public float[] Field2 { get; set; }
}

// Function used to create toy data sets.
private static IEnumerable<DataPoint> GenerateRandomDataPoints(
    int exampleCount, int seed = 0)
{
    var rnd = new Random(seed);
    var data = new List<DataPoint>();
    for (int i = 0; i < exampleCount; ++i)
    {
        // Initialize an example with a random label and an empty feature
        // vector.
        var sample = new DataPoint()
        {
            Label = rnd.Next() % 2 == 0,
            Field0 = new float[featureLength],
            Field1 = new float[featureLength],
            Field2 = new float[featureLength]
        };

        // Fill feature vectors according the assigned label.
        // Notice that features from different fields have different biases
        // and therefore different distributions. In practices such as game
        // recommendation, one may use one field to store features from user
        // profile and another field to store features from game profile.
        for (int j = 0; j < featureLength; ++j)
        {
            var value0 = (float)rnd.NextDouble();
            // Positive class gets larger feature value.
            if (sample.Label)
                value0 += 0.2f;
            sample.Field0[j] = value0;

            var value1 = (float)rnd.NextDouble();
            // Positive class gets smaller feature value.
            if (sample.Label)
                value1 -= 0.2f;
            sample.Field1[j] = value1;

            var value2 = (float)rnd.NextDouble();
            // Positive class gets larger feature value.
            if (sample.Label)
                value2 += 0.8f;
            sample.Field2[j] = value2;
        }

        data.Add(sample);
    }
    return data;
}

IEnumerable<DataPoint> data = GenerateRandomDataPoints(500);
var trainingData = mlContext.Data.LoadFromEnumerable(data);

In [ ]:
var options = new FieldAwareFactorizationMachineTrainer.Options
{
    FeatureColumnName = nameof(DataPoint.Field0),
    ExtraFeatureColumns =
    new[] { nameof(DataPoint.Field1), nameof(DataPoint.Field2) },

    LabelColumnName = nameof(DataPoint.Label),
    LambdaLatent = 0.01f,
    LambdaLinear = 0.001f,
    LatentDimension = 16,
    NumberOfIterations = 50,
    LearningRate = 0.5f
};

var pipeline = mlContext.BinaryClassification.Trainers
    .FieldAwareFactorizationMachine(options);

In [ ]:
var model = pipeline.Fit(trainingData);

In [ ]:
var transformedTrainingData = model.Transform(trainingData);
var metrics = mlContext.BinaryClassification.Evaluate(transformedTrainingData);

In [ ]:
// Function used to show evaluation metrics such as accuracy of predictions.
private static void PrintMetrics(CalibratedBinaryClassificationMetrics metrics)
{
    Console.WriteLine($"Accuracy: {metrics.Accuracy:F2}");
    Console.WriteLine($"AUC: {metrics.AreaUnderRocCurve:F2}");
    Console.WriteLine($"F1 Score: {metrics.F1Score:F2}");
    Console.WriteLine($"Negative Precision: {metrics.NegativePrecision:F2}");

    Console.WriteLine($"Negative Recall: {metrics.NegativeRecall:F2}");
    Console.WriteLine($"Positive Precision: {metrics.PositivePrecision:F2}");

    Console.WriteLine($"Positive Recall: {metrics.PositiveRecall:F2}");
    Console.WriteLine($"Log Loss: {metrics.LogLoss:F2}");
    Console.WriteLine($"Log Loss Reduction: {metrics.LogLossReduction:F2}");
    Console.WriteLine($"Entropy: {metrics.Entropy:F2}");
}

PrintMetrics(metrics);

In [ ]:
// This class defines objects produced by trained model. The trained model
// maps a DataPoint to a Result.
public class Result
{
    // Label.
    public bool Label { get; set; }
    // Predicted label.
    public bool PredictedLabel { get; set; }
    // Predicted score.
    public float Score { get; set; }
    // Probability of belonging to positive class.
    public float Probability { get; set; }
}

var engine = mlContext.Model.CreatePredictionEngine<DataPoint, Result>(model);
foreach (var dataPoint in data.Take(5))
{
    var result = engine.Predict(dataPoint);
    Console.WriteLine($"Actual label: {dataPoint.Label}, " +
        $"predicted label: {result.PredictedLabel}, " +
        $"score of being positive class: {result.Score}, " +
        $"and probability of being positive class: " +
        $"{result.Probability}."
);}